# Assignment #3 - AI Fall 98



در این پروژه قصد پیاده سازی برنامه ای را داریم که بتواند شاعر یک بیت شعر را تشخیص دهد.
ابتدا با بررسی مشخصات و حالات بهترین ویژگی هارا برای تشخیص شاعر پیدا کرده و الگوریتم مربوطه را پیاده سازی می کنیم. 
برای این منظور ۸۰٪ از اطلاعات در اختیار قرار داده شده را به عنوان ترین ـ دیتا بررسی کرده و باقی ۲۰٪ باقیمانده را برای بررسی نگه می داریم.


ابتدا کتابخانه های مورد نیاز و دستورات لودینگ دیتاها را اجرا می کنیم.

سپس برای انتخاب ۸۰٪ دیتای مورد نیاز به صورت رندوم اقدام می کنیم.

In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import matplotlib.pyplot as plt
import math
import re
import numpy as np
import random


file = 'train_test.csv'
file_evaluate = 'evaluate.csv'
Data = pd.read_csv(file)

poems = Data['text']
statuses = Data['label']
numberOfpoems = len(poems)




train_index = random.sample(range(0,numberOfpoems), int(.8 * numberOfpoems))
test_index = list(set([item for item in range(0, numberOfpoems)]) - set(train_index))
hafez_poems_in_train_data = []
saadi_poems_in_train_data = []
total_vocab_in_train_data = {}
hafez_vocab_in_train_data = {}
saadi_vocab_in_train_data = {}


train_index.sort()
test_index.sort()


سپس برای دقیق تر کار کردن و فهم بهتر شعر های هر شاعر را به صورت تفکیک شده ذخیره می کنیم.

پس از آن احتمال تعلق شعر به هر شاعر را نسبت به تعداد اشعار آن شاعر حساب می کنیم

به صورت همزمان تعداد کلمات مجزا را در دیتا ترین به دست می آوریم

In [2]:

for index in train_index:
    poem = poems[index]
    words = poem.split(' ')
    if statuses[index] == 'hafez' :
        hafez_poems_in_train_data.append(poems[index])
    elif statuses[index] == 'saadi' :
        saadi_poems_in_train_data.append(poems[index])
    for word in words:
        if word not in total_vocab_in_train_data:
            total_vocab_in_train_data[word] = 0
        else:
            total_vocab_in_train_data[word]+=1


probability_hafez = (len(hafez_poems_in_train_data) / float(len(train_index)) )
probability_saadi = (len(saadi_poems_in_train_data) / float(len(train_index)) )
unique_words = 0 
unique_words = len(total_vocab_in_train_data)
total_vocab_size_unique_word = len(total_vocab_in_train_data)

num_total_word_vocab_not_unique = 0
for key in total_vocab_in_train_data :
    num_total_word_vocab_not_unique+=total_vocab_in_train_data[key]

در این قسمت با کمک تابع زیر تعداد کلمات مجزا را نیز برای شاعر ها به دست آورده و همینطور تعداد تکرار کلمات در اشعار حافظ و سعدی را به دست می آوریم

In [3]:

def recognizer(poems_td, vocab_td):
    for poem in poems_td:
        words = poem.split(' ')
        for word in words:
            if word not in vocab_td:
                vocab_td[word] = 0
            else:
                vocab_td[word]+=1
    return vocab_td


unique_words = 0 
unique_words = len(total_vocab_in_train_data)
hafez_vocab_in_train_data = recognizer(hafez_poems_in_train_data,hafez_vocab_in_train_data)
saadi_vocab_in_train_data = recognizer(saadi_poems_in_train_data,saadi_vocab_in_train_data)
   

num_total_word_saadi_unique = len(saadi_vocab_in_train_data) 
num_total_word_saadi_not_unique = 0
for key in saadi_vocab_in_train_data :
    num_total_word_saadi_not_unique += saadi_vocab_in_train_data[key]


num_total_word_hafez_unique = len(hafez_vocab_in_train_data) 
num_total_word_hafez_not_unique = 0
for key in hafez_vocab_in_train_data :
    num_total_word_hafez_not_unique+=hafez_vocab_in_train_data[key]


دیتا های به دست آمده به شرح زیر می باشد :

In [4]:
print("Total Number Of Poems in train_test:",numberOfpoems)
print("hafez poems count in 80% train data:",len(hafez_poems_in_train_data))
print("saadi poems count in 80% train data:",len(saadi_poems_in_train_data))
print("probability hafez:" , probability_hafez)
print("probability saadi:" , probability_saadi)
print("num_total_word_vocab_not_unique",num_total_word_vocab_not_unique)
print("total_vocab_size_unique_word",len(total_vocab_in_train_data))
print("num poem in train data",len(train_index))
print("num_total_word_hafez_not_unique:",num_total_word_hafez_not_unique)
print("num_total_word_hafez_unique",num_total_word_hafez_unique)
print("num_total_word_saadi_not_unique:",num_total_word_saadi_not_unique  )
print("num_total_word_saadi_unique",num_total_word_saadi_unique)

Total Number Of Poems in train_test: 20889
hafez poems count in 80% train data: 6722
saadi poems count in 80% train data: 9989
probability hafez: 0.40225001496020585
probability saadi: 0.5977499850397942
num_total_word_vocab_not_unique 109336
total_vocab_size_unique_word 12672
num poem in train data 16711
num_total_word_hafez_not_unique: 43287
num_total_word_hafez_unique 7376
num_total_word_saadi_not_unique: 62124
num_total_word_saadi_unique 9221


حال کافی است که در قسمت نهایی بخش ترین خود به محاسبه احتمال وجود کلمه به شرط انکه شاعرش حافظ یا سعدی باشد می پردازیم .در حقیقت با استفاده از فرمول زیر احتمال را محاسبه می کنیم.

p ( wk | Vj )  = nk + 1 / n + |vocabulary |  لاپلاس

در اینجا سایز لغات را برابر تعداد کلمات مجزا و
عدد
ان را
را تعداد کل کلماتی که در حافظ و سعدی به کار رفته است در نظر می گیریم.

p ( w | hafez ) =  تعداد دفعاتی که کلمه مورد نظر در حافظ آمده +۱/  تعداد کماتی که در حافظ آمده + تعداد کلماتی که در کل آمده 



p ( w | saadi ) =  تعداد دفعاتی که کلمه مورد نظر در سعدی آمده +۱/  تعداد کماتی که در سعدی آمده + تعداد کلماتی که در کل آمده 


براساس فرمول های فوق احتمال کلمه به شرط لغات محسابه می کنیم. 
برای مثال وقتی کلمه ای در بیت های ترین سعدی به کار نرفته باشد ولی در بیت های تست دیتا قرار گرفته باشد حاصل ضرب احتمال ۰ می شود ولی با در نظر گرفتن یک احتمال دیگر صفر نمی شود.


In [5]:
def calculate_probability_conditional(word,saadi_vocab_in_train_data,total_vocab_in_train_data,
                                        num_total_word_saadi_not_unique ,unique_words, condition):
    
    if(condition == "Laplace"):    
        num_word_in_saadi = 0
        if word in saadi_vocab_in_train_data:
            num_word_in_saadi = saadi_vocab_in_train_data[word]
        x =  num_word_in_saadi + 1
        y = float( num_total_word_saadi_not_unique + len(total_vocab_in_train_data))
        return x/y
    

    elif(condition == "Simple"):
        num_word_in_saadi = 0
        if word in saadi_vocab_in_train_data:
            num_word_in_saadi = saadi_vocab_in_train_data[word]
        x = num_word_in_saadi + .3
        y = float(num_total_word_saadi_not_unique + len(total_vocab_in_train_data))
        return x/y


حال در این قسمت با توجه به دیتا ترین کافی است که از ۲۰ درصد باقی مانده برای تست عملکرد استفاده کنیم .

در اینجا سه مقدار را براساس فرمول های داده شده در صورت پروژه بدست خواهیم آورد

حال کافی است در هر اندیس دیتا تست بیاییم هر شعر را بررسی کنیم و  دو مقدار زیر را هم برای حافظ و هم برای سعدی حساب کنیم  و هر کدام که بیشتر شد در حقیقت مدل ما آن را پیش بینی کرده است حال با نتیجه جواب مقایسه می کنیم و مقادیر را آپدیت می کنیم



این تابع هردو حالت لاپلاس و عادی را به اضای ورودی خواسته شده حساب می کند.

In [6]:
def Unit_Test(test_index,
                statuses,
                poems,
                probability_hafez,
                probability_saadi,
                total_vocab_in_train_data,
                unique_words,
                num_total_word_vocab_not_unique,
                hafez_vocab_in_train_data,
                saadi_vocab_in_train_data,
                num_total_word_hafez_unique,
                num_total_word_saadi_unique,
                num_total_word_hafez_not_unique,
                num_total_word_saadi_not_unique):
    
    hafez_poems_in_test_data = []
    saadi_poems_in_test_data = []
    num_total_word_vocab_not_unique = 0
    total_vocab_in_test_data = {}
    hafez_vocab_in_test_data = {}
    saadi_vocab_in_test_data = {}
    correct_detected_hafezes = 0
    detected_hafezes = 0
    correct_detected = 0
    select_op = 0
    num_total_word_hafez_not_unique = 0
    num_total_word_saadi_not_unique = 0

    for index in test_index :
        if statuses[index] == 'hafez' :
            hafez_poems_in_test_data.append(poems[index])
        elif True:
            saadi_poems_in_test_data.append(poems[index])
            
    hafez_numberOfpoems_in_test_data = len(hafez_poems_in_test_data)
    saadi_numberOfpoems_in_test_data = len(saadi_poems_in_test_data)
 

    for index in test_index:
        poem = poems[index]
        words = poem.split(' ')
        for word in words:
            if word not in total_vocab_in_test_data:
                total_vocab_in_test_data[word] = 0
            elif True:
                total_vocab_in_test_data[word]+=1
            
    unique_words = len(total_vocab_in_test_data)

    

    for key in total_vocab_in_test_data :
        num_total_word_vocab_not_unique += total_vocab_in_test_data[key]
    

    hafez_vocab_in_test_data = recognizer(hafez_poems_in_train_data,hafez_vocab_in_train_data)
    saadi_vocab_in_test_data = recognizer(saadi_poems_in_train_data,saadi_vocab_in_train_data)

              
                
    num_total_word_hafez_unique = len(hafez_vocab_in_test_data) 
    for key in hafez_vocab_in_test_data :
        num_total_word_hafez_not_unique += hafez_vocab_in_test_data[key]
    
    num_total_word_saadi_unique = len(saadi_vocab_in_test_data) 

    
    for key in saadi_vocab_in_test_data :
        num_total_word_saadi_not_unique += saadi_vocab_in_test_data[key]

    print("\n\n Unit Test Initialization OutPut :")
    print("hafez poems count in 20% test data:",hafez_numberOfpoems_in_test_data)
    print("saadi poems count in 20% test data:",saadi_numberOfpoems_in_test_data)
    print("num_total_word_vocab_not_unique in 20% test",num_total_word_vocab_not_unique)
    print("total_vocab_size_unique_word in 20 % test",len(total_vocab_in_train_data))
    print("num poem in train data in 20% test",len(test_index))
    print("num_total_word_hafez_not_unique in 20% test:",num_total_word_hafez_not_unique)
    print("num_total_word_hafez_unique in 20% test: ",num_total_word_hafez_unique)
    print("num_total_word_saadi_not_unique in 20% test:",num_total_word_saadi_not_unique  )
    print("num_total_word_saadi_unique in 20 % test",num_total_word_saadi_unique)
    print("Unit Test Initialization OutPuts Ends \n")
        
    
    all_hafezes = len(hafez_poems_in_test_data)
    total = len(test_index)
    
    
    select_op = int(input("\n\n\n '1'.... Laplace \n '2' ....Simple \n\n\n"))
    
    for index in test_index:
        poem = poems[index]
        words = poem.split(' ')
        p_hafez = probability_hafez
        p_saadi = probability_saadi
        for word in words:
            if(select_op == 2):
                p_hafez *= calculate_probability_conditional(word,hafez_vocab_in_train_data,
                                                                total_vocab_in_train_data,
                                                                num_total_word_hafez_not_unique ,
                                                                len(total_vocab_in_train_data),"Simple")
                
                p_saadi *= calculate_probability_conditional(word,saadi_vocab_in_train_data,
                                                            total_vocab_in_train_data,
                                                                num_total_word_saadi_not_unique ,
                                                            len(total_vocab_in_train_data),"Simple"  )
            if(select_op == 1): 
                p_hafez *= calculate_probability_conditional(word,hafez_vocab_in_train_data,
                                                           total_vocab_in_train_data,
                                                            num_total_word_hafez_not_unique ,
                                                            len(total_vocab_in_train_data),"Laplace")
                p_saadi *= calculate_probability_conditional(word,saadi_vocab_in_train_data,total_vocab_in_train_data,
                                                            num_total_word_saadi_not_unique , len(total_vocab_in_train_data),"Laplace")

        if p_hafez > p_saadi:

            detected_hafezes += 1
            if statuses[index] == 'hafez':
                correct_detected_hafezes += 1
                correct_detected += 1
          
        ##model decide saadi
        elif p_hafez < p_saadi:
            if statuses[index] =='saadi':
                correct_detected+=1

    recall = correct_detected_hafezes/float(all_hafezes)
    precision =  correct_detected_hafezes/ float(detected_hafezes)  
    accuracy = correct_detected / float(total)
            
    if(select_op == 2):
        print("Recall in simple:",recall)
        print("Precision in simple:",precision)
        print("Accuracy in simple:",accuracy)
    if (select_op == 1):
        print("Recall in Laplace:",recall)
        print("Precision in Laplace:",precision)
        print("Accuracy in Laplace:",accuracy)



Unit_Test(test_index,
            statuses,
            poems,
            probability_hafez,
            probability_saadi,
            total_vocab_in_train_data,
            len(total_vocab_in_train_data),
            num_total_word_vocab_not_unique,
            hafez_vocab_in_train_data,
            saadi_vocab_in_train_data,
            num_total_word_hafez_unique,
            num_total_word_saadi_unique,
            num_total_word_hafez_not_unique,
            num_total_word_saadi_not_unique)
        



 Unit Test Initialization OutPut :
hafez poems count in 20% test data: 1696
saadi poems count in 20% test data: 2482
num_total_word_vocab_not_unique in 20% test 24776
total_vocab_size_unique_word in 20 % test 12672
num poem in train data in 20% test 4178
num_total_word_hafez_not_unique in 20% test: 93950
num_total_word_hafez_unique in 20% test:  7376
num_total_word_saadi_not_unique in 20% test: 133469
num_total_word_saadi_unique in 20 % test 9221
Unit Test Initialization OutPuts Ends 




 '1'.... Laplace 
 '2' ....Simple 


2
Recall in simple: 0.7246462264150944
Precision in simple: 0.7595797280593325
Accuracy in simple: 0.7951172809956917


Recall in Laplace: 0.7393681652490887


Precision in Laplace: 0.7507711289327575


Accuracy in Laplace: 0.8006223073240785

اطلاعات به دست آمده از تابع برای هر دو حالت ساده و لاپلاس به صورت بالا است

در پایان به ساخت فایل خواسته شده می پردازیم.

In [7]:

def make_evalute_file( evalute_filename ,
                    probability_hafez,
                    probability_saadi,
                    total_vocab_in_train_data,
                    unique_words,
                    num_total_word_vocab_not_unique,
                    hafez_vocab_in_train_data,
                    saadi_vocab_in_train_data,
                    num_total_word_hafez_unique,
                    num_total_word_saadi_unique,
                    num_total_word_hafez_not_unique,
                    num_total_word_saadi_not_unique):
    
    Data = pd.read_csv(evalute_filename)
    index = Data['id']
    poems = Data['text']
    statuses = []
    for poem in poems:
        words = poem.split(' ')
        p_hafez = probability_hafez
        p_saadi = probability_saadi
        for word in words:
            p_hafez *= calculate_probability_conditional(word,hafez_vocab_in_train_data,
                                                           total_vocab_in_train_data,
                                                            num_total_word_hafez_not_unique ,
                                                            len(total_vocab_in_train_data),"Laplace")
            p_saadi *= calculate_probability_conditional(word,saadi_vocab_in_train_data,total_vocab_in_train_data,
                                        num_total_word_saadi_not_unique , len(total_vocab_in_train_data),"Laplace")
            
        if p_hafez > p_saadi:
            statuses.append('hafez')
        elif p_hafez < p_saadi:
            statuses.append('saadi')
    Data.insert(2,'label',statuses)
    print(Data.head(20))
    Data.to_csv("output.csv")

In [8]:
make_evalute_file(file_evaluate ,
                    probability_hafez,
                    probability_saadi,
                    total_vocab_in_train_data,
                    len(total_vocab_in_train_data),
                    num_total_word_vocab_not_unique,
                    hafez_vocab_in_train_data,
                    saadi_vocab_in_train_data,
                    num_total_word_hafez_unique,
                    num_total_word_saadi_unique,
                    num_total_word_hafez_not_unique,
                    num_total_word_saadi_not_unique)

    id                                   text  label
0    1       ور بی تو بامداد کنم روز محشر است  saadi
1    2        ساقی بیار جامی کز زهد توبه کردم  hafez
2    3          مرا هرآینه خاموش بودن اولی‌تر  saadi
3    4   تو ندانی که چرا در تو کسی خیره بماند  saadi
4    5         کاینان به دل ربودن مردم معینند  saadi
5    6      جان باختن آسان است اندر نظرت لیکن  saadi
6    7          گر خاک پای دوست خداوند شوق را  saadi
7    8        بی تو در دامن گلزار نخفتم یک شب  saadi
8    9      پرده بردار که هوش از تن عاقل برود  saadi
9   10         چرا نه در پی عزم دیار خود باشم  hafez
10  11        که زندگانی او در هلاک بودن اوست  saadi
11  12     که نه هر کو ورقی خواند معانی دانست  hafez
12  13  زلف آشفته و خوی کرده و خندان لب و مست  hafez
13  14         نسیم موی تو پیوند جان آگه ماست  hafez
14  15       من همان روز که روی تو بدیدم گفتم  saadi
15  16             اری اسامر لیلای لیله القمر  hafez
16  17                تو وفا گر کنی و گر نکنی  saadi
17  18       سودا مپز که آتش غم در دل تو نیست 